In [2]:
import matplotlib
matplotlib.use('TkAgg') #crashes when using %matplotlib qt

In [3]:
import numpy
from matplotlib import pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as Tk
import PySpin
from datetime import datetime
import os
import math

In [6]:
class LineDrawer(object):
    lines = []
    def draw_line(self, canvas, image):
        ax = plt.gca()
        xy = plt.ginput(2)        
        x = [p[0] for p in xy]
        y = [p[1] for p in xy]
        line = plt.plot(x,y, 'r')
        ax.figure.canvas.draw()
        self.lines.append(line)
        distance = math.sqrt((x[1] - x[0])**2 + (y[1] - y[0])**2)
        distance_s = '%4.3f' % distance
        bbox_props = dict(boxstyle="round,pad=0.3", facecolor='beige', alpha=1.0, lw=2)
        fig = plt.gcf()
        size = fig.get_size_inches()*fig.dpi # size in pixels
        t = ax.text(.5*numpy.shape(image)[1], 0, 'The distance is '+distance_s+' px', ha="center", va="center", size=20,bbox=bbox_props)
        canvas.draw()
        print('The distance in pixels is %4.3f' % distance)
        
def list_object_parameters(obj):
    import inspect
    return [a[0] for a in inspect.getmembers(obj)]

def is_param_in_object(param, obj):
    import inspect
    params = [a[0] for a in inspect.getmembers(obj)]
    return param in params

def set_custom_cam_settings(cam, **kwargs):
    params = list_object_parameters(cam)
    try:
        result = True
        for k, v in kwargs.items():
            if (k in params) and (getattr(cam,k).GetAccessMode() == PySpin.RW):
                if k == 'Width':
                    getattr(cam, k).SetValue(v)
                    print('Setting', k, '=', v)
                    cam.OffsetX.SetValue(int((5472-v)/2))
                    print('Setting OffsetX =', (5472-v)/2)
                elif k == 'Height':
                    getattr(cam, k).SetValue(v)
                    print('Setting', k, '=', v)
                    cam.OffsetY.SetValue(int((3648-v)/2))
                    print('Setting OffsetY =', (3648-v)/2)
                else: 
                    getattr(cam, k).SetValue(v)
                    print('Setting', k, '=', v)
            else:
                print(k, 'is not a parameter of this object or not accessible')
                result = False
    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
    return result

class LineDrawer(object):
    lines = []
    def draw_line(self, canvas, image):
        ax = plt.gca()
        xy = plt.ginput(2)        
        x = [p[0] for p in xy]
        y = [p[1] for p in xy]
        line = plt.plot(x,y, 'r')
        ax.figure.canvas.draw()
        self.lines.append(line)
        distance = math.sqrt((x[1] - x[0])**2 + (y[1] - y[0])**2)
        distance_s = '%4.3f' % distance
        bbox_props = dict(boxstyle="round,pad=0.3", facecolor='beige', alpha=1.0, lw=2)
        fig = plt.gcf()
        size = fig.get_size_inches()*fig.dpi # size in pixels
        t = ax.text(.5*numpy.shape(image)[1], 0, 'The distance is '+distance_s+' px', ha="center", va="center", size=20,bbox=bbox_props)
        canvas.draw()
        print('The distance in pixels is %4.3f' % distance)

def app(image):
    root = Tk.Tk()
    root.wm_title("minimal example")

    image=image
    fig = plt.figure(figsize=(10,10))
    # im = plt.imshow(image) # later use a.set_data(new_data)
    plt.imshow(image, cmap='gray')
    ax = plt.gca()
    ax.set_xticklabels([]) 
    ax.set_yticklabels([]) 

    # a tk.DrawingArea
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)

    ld = LineDrawer()
    ld.draw_line(canvas, image) # here you click on the plot

    def quit(*args):
        root.quit()     
        root.destroy() 

    button_quit = Tk.Button(master = root, text = 'Quit', command = quit)
    root.protocol("WM_DELETE_WINDOW", quit)

    button_quit.pack(side=Tk.LEFT)

    Tk.mainloop()
    
def acquire_images(cam, num_images):
    try:
        # Set acquisition mode to continuous
        if cam.AcquisitionMode.GetAccessMode() != PySpin.RW:
            print('Unable to set acquisition mode to continuous. Aborting...')
        cam.AcquisitionMode.SetValue(PySpin.AcquisitionMode_Continuous)
        # Begin acquiring images
        cam.BeginAcquisition()
        print('Acquiring images...')
        
        for i in range(num_images):
            try:
                # Retrieve next received image and ensure image completion
                image_result = cam.GetNextImage()
                if image_result.IsIncomplete():
                    print('Image incomplete with image status %d...' % image_result.GetImageStatus())
                else:
                    image = image_result.GetNDArray()
                    # Release image
                    image_result.Release()
                    return image
            except PySpin.SpinnakerException as ex:
                print('Error: %s' % ex)
        
        # End acquisition
        cam.EndAcquisition()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)

def run_cam(params, num_images=1):
    result = True
    # Get system
    system = PySpin.System.GetInstance()
    # Get camera list
    cam_list = system.GetCameras()
    num_cameras = cam_list.GetSize()
    print('Number of cameras detected: %d' % num_cameras)
    # Finish if there are no cameras
    if num_cameras == 0:
        # Clear camera list before releasing system
        cam_list.Clear()
        # Release system instance
        system.ReleaseInstance()
        print('Not enough cameras!')
        return False
    
    cam = cam_list.GetByIndex(0)
    try: 
        # Initialize camera
        cam.Init()
        # Configure camera settings
        set_custom_cam_settings(cam, **params)
        # Acquire images
        image = acquire_images(cam, num_images)
        app(image)
        
        set_custom_cam_settings(cam, ExposureAuto=True, GainAuto=True)
        # Deinitialize camera
        cam.DeInit()
        return result

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False
    
    del cam
    cam_list.Clear()
    system.ReleaseInstance()
    return result

In [4]:
#max width = 5472
#max height = 3648

params = {
    'Width': 912,
    'Height': 1824,
    'AcquisitionFrameRateEnable': True,
    'AcquisitionFrameRate': 3,
    'ExposureAuto': False,
    'ExposureTime': 15000,
    'GainAuto': False,
    'Gain': 6.29
}

In [ ]:
run_cam(params, 1)

Number of cameras detected: 1
Setting Width = 912
Setting OffsetX = 2280.0
Setting Height = 1824
Setting OffsetY = 912.0
Setting AcquisitionFrameRateEnable = True
Setting AcquisitionFrameRate = 3
Setting ExposureAuto = False
Setting ExposureTime = 15000
Setting GainAuto = False
Setting Gain = 6.29
Acquiring images...
The distance in pixels is 513.905
Setting ExposureAuto = True
Setting GainAuto = True
Error: Spinnaker: Can't de-initialize camera. Camera is still streaming. [-1004]


In [6]:
import os

In [7]:
os.getcwd()

'C:\\Users\\bwilks\\Documents\\usb_cam\\USB_CAM'

In [21]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 8459-6FB3

 Directory of C:\Users\bwilks\documents\usb_cam\usb_cam

01/09/2019  09:14 PM    <DIR>          .
01/09/2019  09:14 PM    <DIR>          ..
01/09/2019  08:09 PM    <DIR>          .ipynb_checkpoints
01/04/2019  04:49 PM    <DIR>          docs
01/07/2019  02:38 PM    <DIR>          Examples
01/04/2019  04:49 PM            12,553 README.txt
01/09/2019  08:05 PM             5,171 run_cam.py
01/09/2019  04:18 PM    <DIR>          SFM
01/04/2019  04:49 PM        32,366,794 spinnaker_python-1.19.0.22-cp36-cp36m-win_amd64.whl
01/09/2019  08:12 PM             1,011 test.py
01/09/2019  09:09 PM            52,167 Untitled.ipynb
01/09/2019  08:29 PM           143,112 usb_cam_acquisition.ipynb
01/09/2019  09:14 PM            13,625 usb_cam_matploblib.ipynb
               7 File(s)     32,594,433 bytes
               6 Dir(s)  187,445,182,464 bytes free


In [23]:
filename

'C:\\Users\\bwilks\\volume.png'

In [ ]:
class LineDrawer(object):
    lines = []
    def draw_line(self):
        ax = plt.gca()
        xy = plt.ginput(2)        
#         x = [p[0] for p in xy]
#         y = [p[1] for p in xy]
#         line = plt.plot(x,y)
#         ax.figure.canvas.draw()
#         self.lines.append(line)
#         distance = math.sqrt((x[1] - x[0])**2 + (y[1] - y[0])**2)
#         distance_s = '%4.3f' % distance
#         bbox_props = dict(boxstyle="round,pad=0.3", ec="b", lw=2)
#         t = ax.text(0, 0, 'The distance is '+distance_s+' px', ha="center", va="center", size=15,bbox=bbox_props)
# #         ax.annotate('The distance is '+distance_s+' px', (0,0), horizontalalignment='right', verticalalignment='top')
#         print('The distance in pixels is %4.3f' % distance)
fig, current_ax = plt.subplots(figsize=(10,10))             
ld = LineDrawer()
ld.draw_line() # here you click on the plot

In [ ]:
fig, current_ax = plt.subplots(figsize=(10,10))             
ld = LineDrawer()
ld.draw_line() # here you click on the plot

In [ ]:
if cam.Width.GetAccessMode() == PySpin.RW and cam.Width.GetInc() != 0 and cam.Width.GetMax != 0:
    cam.Width.SetValue(912)
else:
    print('Width not available...')
    result = False

In [11]:
system = PySpin.System.GetInstance()
# Get camera list
cam_list = system.GetCameras()
num_cameras = cam_list.GetSize()

print('Number of cameras detected: %d' % num_cameras)
# Finish if there are no cameras
if num_cameras == 0:
    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()

    print('Not enough cameras!')
    input('Done! Press Enter to exit...')


cam = cam_list.GetByIndex(0)
cam.Init()

Number of cameras detected: 1


In [15]:
cam.DeInit()

In [ ]:
cam.Width()

In [ ]:
??cam.AcquisitionStop()

In [7]:
list_object_parameters(cam)

['AasRoiEnable',
 'AasRoiHeight',
 'AasRoiOffsetX',
 'AasRoiOffsetY',
 'AasRoiWidth',
 'AcquisitionAbort',
 'AcquisitionArm',
 'AcquisitionBurstFrameCount',
 'AcquisitionFrameCount',
 'AcquisitionFrameRate',
 'AcquisitionFrameRateEnable',
 'AcquisitionLineRate',
 'AcquisitionMode',
 'AcquisitionResultingFrameRate',
 'AcquisitionStart',
 'AcquisitionStatus',
 'AcquisitionStatusSelector',
 'AcquisitionStop',
 'ActionDeviceKey',
 'ActionGroupKey',
 'ActionGroupMask',
 'ActionQueueSize',
 'ActionSelector',
 'ActionUnconditionalMode',
 'AdaptiveCompressionEnable',
 'AdcBitDepth',
 'AutoAlgorithmSelector',
 'AutoExposureControlLoopDamping',
 'AutoExposureControlPriority',
 'AutoExposureEVCompensation',
 'AutoExposureExposureTimeLowerLimit',
 'AutoExposureExposureTimeUpperLimit',
 'AutoExposureGainLowerLimit',
 'AutoExposureGainUpperLimit',
 'AutoExposureGreyValueLowerLimit',
 'AutoExposureGreyValueUpperLimit',
 'AutoExposureLightingMode',
 'AutoExposureMeteringMode',
 'AutoExposureTargetGrey

In [10]:
?? cam.PixelFormat.

Object ` cam.PixelFormat.` not found.
